<a href="https://colab.research.google.com/github/martinimass/openni2-frame-extraction/blob/main/Openni2_frame_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Openni2 Frame Extraction Example

## Dependency

In [11]:
!pip install openni

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/bin/python -m pip install --upgrade pip' command.


## Utility

In [9]:
import os
import sys
import cv2
from openni import openni2
import argparse
import numpy as np

def openDevice(video_path):
    try:
        if sys.platform == "win32":
            libpath = "lib/Windows"
        else:
            libpath = "lib/Linux"
        openni2.initialize(libpath)
        dev = openni2.Device.open_file(video_path)
        pbs = openni2.PlaybackSupport(dev)

        pbs.set_repeat_enabled(False)
        pbs.set_speed(-1.0)

        return dev,pbs
    except Exception as ex:
        print(ex)
        raise Exception("Initialization Error")

def processDepth(dev,pbs,interval,dst):
    try:
        depth_stream = dev.create_depth_stream()
        depth_stream.start()
        print("Depth frames: " + str(depth_stream.get_number_of_frames()))
        for i in range(1,depth_stream.get_number_of_frames()+1,interval):
            pbs.seek(depth_stream,i)
            frame_depth = depth_stream.read_frame()
            print(i, frame_depth.timestamp)
            frame_depth_data = frame_depth.get_buffer_as_uint16()

            depth_array = np.ndarray((frame_depth.height, frame_depth.width),dtype=np.uint16,buffer=frame_depth_data)

            #Depth frame conversion: from 16bit to 8bit
            depth_scale_factor = 0.05
            depth_scale_beta_factor = 0
            depth_uint8 = depth_array * depth_scale_factor + depth_scale_beta_factor
            depth_uint8[depth_uint8 > 255] = 255
            depth_uint8[depth_uint8 < 0] = 0
            depth_uint8 = depth_uint8.astype('uint8')

#             cv2.imwrite(dst + "/" + f"{frame_depth.frameIndex:04d}" + "_16bit.png",depth_array)
#             cv2.imwrite(dst + "/" + f"{frame_depth.frameIndex:04d}" + "_8bit.png",depth_uint8)
#             print(dst + "/" + f"{frame_depth.frameIndex:04d}" + "_16bit.png")
#             print(dst + "/" + f"{frame_depth.frameIndex:04d}" + "_8bit.png")            
        depth_stream.close()
        print("All depth frames extracted")
    except Exception as ex:
        print("Exception: {}".format(ex))

def processColor(dev,pbs,interval,dst):
    try:
        color_stream = dev.create_color_stream()
        color_stream.start()
        print("Color frames: " + str(color_stream.get_number_of_frames()))
        for i in range(1,color_stream.get_number_of_frames()+1,interval):
            pbs.seek(color_stream,i)
            frame_color = color_stream.read_frame()
            print(i, frame_color.timestamp)
            frame_color_data = frame_color.get_buffer_as_uint8()
            color_array = np.ndarray((frame_color.height, frame_color.width, 3),dtype=np.uint8,buffer=frame_color_data)
            color_array = cv2.cvtColor(color_array, cv2.COLOR_BGR2RGB)
#             cv2.imwrite(dst + "/" + f"{frame_color.frameIndex:04d}" + "_color.png",color_array)
        color_stream.close()
        print("All color frames extracted")
    except Exception as ex:
        print(ex)

## Extraction

In [5]:
import boto3
s3_bucket = "tvpr-dataset"

s3 = boto3.resource('s3')
bucket = s3.Bucket(s3_bucket)
print(bucket.name)

tvpr-dataset


In [6]:
source_prefix = 'oni/'

objects_oni = bucket.objects.filter(Prefix=source_prefix)
objects_oni = [obj for obj in objects_oni if obj.key.endswith(".oni")]
print(objects_oni)
print([obj.key for obj in objects_oni])

[s3.ObjectSummary(bucket_name='tvpr-dataset', key='oni/g001.oni'), s3.ObjectSummary(bucket_name='tvpr-dataset', key='oni/g002.oni'), s3.ObjectSummary(bucket_name='tvpr-dataset', key='oni/g003.oni'), s3.ObjectSummary(bucket_name='tvpr-dataset', key='oni/g004.oni'), s3.ObjectSummary(bucket_name='tvpr-dataset', key='oni/g005.oni'), s3.ObjectSummary(bucket_name='tvpr-dataset', key='oni/g006.oni'), s3.ObjectSummary(bucket_name='tvpr-dataset', key='oni/g007.oni'), s3.ObjectSummary(bucket_name='tvpr-dataset', key='oni/g008.oni'), s3.ObjectSummary(bucket_name='tvpr-dataset', key='oni/g009.oni'), s3.ObjectSummary(bucket_name='tvpr-dataset', key='oni/g010.oni'), s3.ObjectSummary(bucket_name='tvpr-dataset', key='oni/g011.oni'), s3.ObjectSummary(bucket_name='tvpr-dataset', key='oni/g012.oni'), s3.ObjectSummary(bucket_name='tvpr-dataset', key='oni/g013.oni'), s3.ObjectSummary(bucket_name='tvpr-dataset', key='oni/g014.oni'), s3.ObjectSummary(bucket_name='tvpr-dataset', key='oni/g015.oni'), s3.Object

In [10]:
interval = 1            # Interval
temp_dir = "temp/"

for object_oni in objects_oni[6:8]:
    print(object_oni.key)
    
    filename = temp_dir + "/" + object_oni.key
    print(filename)
    
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    if not os.path.exists(filename):
        bucket.download_file(Filename=filename, Key=object_oni.key)
        
    video_path = filename
    dst = temp_dir + "/frames/"  + object_oni.key.split("/")[-1] + "/"     # Destination Folder
    print(dst)
    os.makedirs(dst, exist_ok=True)
    print("Directory ",dst ," Created ")
    
    dev,pbs = openDevice(video_path.encode('utf-8'))
    pbs.set_repeat_enabled(True)

    if dev.has_sensor(openni2.SENSOR_COLOR):
        print("Color Stream found")
        processColor(dev,pbs,interval,dst)

    if dev.has_sensor(openni2.SENSOR_DEPTH):
        print("Depth Stream found")
        processDepth(dev,pbs,interval,dst)

oni/g007.oni
temp//oni/g007.oni
temp//frames/g007.oni/
Directory  temp//frames/g007.oni/  Created 
Color Stream found
Color frames: 3721
1 0
2 63213
3 96729
4 130245
5 163761
6 197277
7 230793
8 264309
9 297825
10 331341
11 364857
12 398373
13 431889
14 465405
15 498921
16 532437
17 565953
18 599469
19 632985
20 666501
21 700017
22 733533
23 767049
24 800565
25 834081
26 867597
27 901113
28 934629
29 968145
30 1001661
31 1035177
32 1068693
33 1102209
34 1135725
35 1169241
36 1202757
37 1236273
38 1269789
39 1303305
40 1336821
41 1370337
42 1403853
43 1437369
44 1470885
45 1504401
46 1537917
47 1571433
48 1604949
49 1638465
50 1671981
51 1705497
52 1739013
53 1772529
54 1806045
55 1839561
56 1873077
57 1906593
58 1940109
59 1973625
60 2007141
61 2040657
62 2074173
63 2107689
64 2141205
65 2174721
66 2208237
67 2241753
68 2275269
69 2308785
70 2342301
71 2375817
72 2409333
73 2442849
74 2476365
75 2509881
76 2543397
77 2576913
78 2610429
79 2643945
80 2677461
81 2710977
82 2744493
83 277

667 22351353
668 22384869
669 22418385
670 22451901
671 22485417
672 22518933
673 22552449
674 22585965
675 22619481
676 22652997
677 22686513
678 22720029
679 22753545
680 22787061
681 22820577
682 22854093
683 22887609
684 22921125
685 22954641
686 22988157
687 23021673
688 23055189
689 23088705
690 23122221
691 23155737
692 23189253
693 23222769
694 23256285
695 23289801
696 23323317
697 23356833
698 23390349
699 23423865
700 23457381
701 23490897
702 23524413
703 23557929
704 23591445
705 23624961
706 23658477
707 23691993
708 23725509
709 23759025
710 23792541
711 23826057
712 23859573
713 23893089
714 23926605
715 23960121
716 23993637
717 24027153
718 24060669
719 24094185
720 24127701
721 24161217
722 24194733
723 24228249
724 24261765
725 24295281
726 24328797
727 24362313
728 24395829
729 24429345
730 24462861
731 24496377
732 24529893
733 24563409
734 24596925
735 24630441
736 24663957
737 24697473
738 24730989
739 24764505
740 24798021
741 24831537
742 24865053
743 24898569

1305 43734561
1306 43768077
1307 43801593
1308 43835109
1309 43868625
1310 43902141
1311 43935657
1312 43969173
1313 44002689
1314 44036205
1315 44069721
1316 44103237
1317 44136753
1318 44170269
1319 44203785
1320 44237301
1321 44270817
1322 44304333
1323 44337849
1324 44371365
1325 44404881
1326 44438397
1327 44471913
1328 44505429
1329 44538945
1330 44572461
1331 44605977
1332 44639493
1333 44673009
1334 44706525
1335 44740041
1336 44773557
1337 44807073
1338 44840589
1339 44874105
1340 44907621
1341 44941137
1342 44974653
1343 45008169
1344 45041685
1345 45075201
1346 45108717
1347 45142233
1348 45175749
1349 45209265
1350 45242781
1351 45276297
1352 45309813
1353 45343329
1354 45376845
1355 45410361
1356 45443877
1357 45477393
1358 45510909
1359 45544425
1360 45577941
1361 45611457
1362 45644973
1363 45678489
1364 45712005
1365 45745521
1366 45779037
1367 45812553
1368 45846069
1369 45879585
1370 45913101
1371 45946617
1372 45980133
1373 46013649
1374 46047165
1375 46080681
1376 4

1943 65218317
1944 65251833
1945 65285349
1946 65318865
1947 65352381
1948 65385897
1949 65419413
1950 65452929
1951 65486445
1952 65519961
1953 65553477
1954 65586993
1955 65620509
1956 65654025
1957 65687541
1958 65788089
1959 65821605
1960 65855121
1961 65888637
1962 65922153
1963 65955669
1964 65989185
1965 66022701
1966 66056217
1967 66089733
1968 66123249
1969 66156765
1970 66190281
1971 66223797
1972 66257313
1973 66290829
1974 66324345
1975 66357861
1976 66391377
1977 66424893
1978 66458409
1979 66491925
1980 66525441
1981 66558957
1982 66592473
1983 66625989
1984 66659505
1985 66693021
1986 66726537
1987 66760053
1988 66793569
1989 66827085
1990 66860601
1991 66894117
1992 66927633
1993 66961149
1994 66994665
1995 67028181
1996 67061697
1997 67095213
1998 67128729
1999 67162245
2000 67195761
2001 67229277
2002 67262793
2003 67296309
2004 67329825
2005 67363341
2006 67396857
2007 67430373
2008 67463889
2009 67497405
2010 67530921
2011 67564437
2012 67597953
2013 67631469
2014 6

2599 87271845
2600 87305361
2601 87338877
2602 87372393
2603 87405909
2604 87439425
2605 87472941
2606 87506457
2607 87539973
2608 87573489
2609 87607005
2610 87640521
2611 87674037
2612 87707553
2613 87741069
2614 87774585
2615 87808101
2616 87841617
2617 87875133
2618 87908649
2619 87942165
2620 87975681
2621 88009197
2622 88042713
2623 88076229
2624 88109745
2625 88143261
2626 88176777
2627 88210293
2628 88243809
2629 88277325
2630 88310841
2631 88344357
2632 88377873
2633 88411389
2634 88444905
2635 88478421
2636 88511937
2637 88545453
2638 88578969
2639 88612485
2640 88646001
2641 88679517
2642 88713033
2643 88746549
2644 88780065
2645 88813581
2646 88847097
2647 88880613
2648 88914129
2649 88947645
2650 88981161
2651 89014677
2652 89048193
2653 89081709
2654 89115225
2655 89148741
2656 89182257
2657 89215773
2658 89249289
2659 89282805
2660 89316321
2661 89349837
2662 89383353
2663 89416869
2664 89450385
2665 89483901
2666 89517417
2667 89550933
2668 89584449
2669 89617965
2670 8

...

1477 49599860
1478 49633376
1479 49666892
1480 49700408
1481 49733924
1482 49767440
1483 49800956
1484 49834472
1485 49867988
1486 49901504
1487 49935020
1488 49968536
1489 50002052
1490 50035568
1491 50069084
1492 50102600
1493 50136116
1494 50169632
1495 50203148
1496 50236664
1497 50270180
1498 50303696
1499 50337212
1500 50370728
1501 50404244
1502 50437760
1503 50471276
1504 50504792
1505 50538308
1506 50571824
1507 50605340
1508 50638856
1509 50672372
1510 50705888
1511 50739404
1512 50772920
1513 50806436
1514 50839952
1515 50873468
1516 50906984
1517 50940500
1518 50974016
1519 51007532
1520 51041048
1521 51074564
1522 51108080
1523 51141596
1524 51175112
1525 51208628
1526 51242144
1527 51275660
1528 51309176
1529 51342692
1530 51376208
1531 51409724
1532 51443240
1533 51476756
1534 51510272
1535 51543788
1536 51577304
1537 51610820
1538 51644336
1539 51677852
1540 51711368
1541 51744884
1542 51778400
1543 51811916
1544 51845432
1545 51878948
1546 51912464
1547 51945980
1548 5

2128 71418776
2129 71452292
2130 71485808
2131 71519324
2132 71552840
2133 71586356
2134 71619872
2135 71653388
2136 71686904
2137 71720420
2138 71753936
2139 71787452
2140 71820968
2141 71854484
2142 71888000
2143 71921516
2144 71955032
2145 71988548
2146 72022064
2147 72055580
2148 72089096
2149 72122612
2150 72156128
2151 72189644
2152 72223160
2153 72256676
2154 72290192
2155 72323708
2156 72357224
2157 72390740
2158 72424256
2159 72457772
2160 72491288
2161 72524804
2162 72558320
2163 72591836
2164 72625352
2165 72658868
2166 72692384
2167 72725900
2168 72759416
2169 72792932
2170 72826448
2171 72859964
2172 72893480
2173 72926996
2174 72960512
2175 72994028
2176 73027544
2177 73061060
2178 73094576
2179 73128092
2180 73161608
2181 73195124
2182 73228640
2183 73262156
2184 73295672
2185 73329188
2186 73362704
2187 73396220
2188 73429736
2189 73463252
2190 73496768
2191 73530284
2192 73563800
2193 73597316
2194 73630832
2195 73664348
2196 73697864
2197 73731380
2198 73764896
2199 7

559 18726404
560 18759836
561 18793269
562 18826702
563 18860135
564 18893567
565 18927000
566 18960433
567 18993866
568 19027298
569 19060731
570 19094164
571 19127597
572 19161029
573 19194494
574 19227958
575 19261486
576 19295014
577 19328574
578 19362134
579 19395757
580 19429380
581 19463004
582 19496627
583 19530250
584 19563873
585 19597497
586 19631120
587 19664743
588 19698366
589 19731990
590 19765613
591 19799236
592 19832859
593 19866483
594 19900106
595 19933729
596 19967352
597 20000976
598 20034599
599 20068222
600 20101845
601 20135437
602 20169028
603 20202556
604 20236084
605 20269581
606 20303077
607 20336510
608 20369942
609 20403375
610 20436808
611 20470241
612 20503673
613 20537106
614 20570539
615 20603972
616 20637404
617 20670837
618 20704270
619 20737703
620 20771135
621 20804568
622 20838001
623 20871434
624 20904866
625 20938299
626 20971732
627 21005165
628 21038597
629 21072030
630 21105463
631 21138896
632 21172328
633 21205761
634 21239194
635 21272627

1242 41751218
1243 41784841
1244 41818464
1245 41852088
1246 41885711
1247 41919334
1248 41952957
1249 41986581
1250 42020204
1251 42053827
1252 42087450
1253 42121074
1254 42154697
1255 42188320
1256 42221943
1257 42255567
1258 42289190
1259 42322813
1260 42356436
1261 42390060
1262 42423651
1263 42457243
1264 42490771
1265 42524299
1266 42557795
1267 42591291
1268 42624787
1269 42658284
1270 42691780
1271 42725244
1272 42758709
1273 42792142
1274 42825574
1275 42858975
1276 42892376
1277 42925777
1278 42959178
1279 42992579
1280 43025980
1281 43059381
1282 43092782
1283 43126183
1284 43159584
1285 43192985
1286 43226386
1287 43259787
1288 43293188
1289 43326589
1290 43359990
1291 43393391
1292 43426792
1293 43460193
1294 43493594
1295 43527027
1296 43560460
1297 43593956
1298 43627452
1299 43661012
1300 43694572
1301 43728132
1302 43761723
1303 43795315
1304 43828906
1305 43862498
1306 43896089
1307 43929681
1308 43963272
1309 43996895
1310 44030519
1311 44064142
1312 44097765
1313 4

1915 64376077
1916 64409542
1917 64443006
1918 64476471
1919 64509935
1920 64543400
1921 64576864
1922 64610329
1923 64643793
1924 64677258
1925 64710722
1926 64744187
1927 64777651
1928 64811116
1929 64844580
1930 64878045
1931 64911509
1932 64944974
1933 64978438
1934 65011903
1935 65045367
1936 65078832
1937 65112296
1938 65145761
1939 65179225
1940 65212690
1941 65246154
1942 65279619
1943 65313083
1944 65346548
1945 65380012
1946 65413477
1947 65446941
1948 65480406
1949 65513870
1950 65547335
1951 65580799
1952 65614296
1953 65647792
1954 65681352
1955 65714911
1956 65748503
1957 65782094
1958 65815718
1959 65849341
1960 65882964
1961 65916587
1962 65950211
1963 65983834
1964 66017457
1965 66051080
1966 66084704
1967 66118327
1968 66151950
1969 66185573
1970 66219197
1971 66252820
1972 66286443
1973 66320066
1974 66353690
1975 66387313
1976 66420936
1977 66454559
1978 66488151
1979 66521742
1980 66555270
1981 66588798
1982 66622295
1983 66655791
1984 66689255
1985 66722720
1986 6

## Upload

In [39]:
import glob

for object_oni in objects_oni[6:8]:
    dst = temp_dir + "/frames/"  + object_oni.key.split("/")[-1] + "/"

    files = glob.glob(f"{dst}/*")
    for file in files:
        key = file.replace("temp//", "")
        bucket.upload_file(Filename=file, Key=key)
    print(files[:100])

['temp//frames/g007.oni/2381_8bit.png', 'temp//frames/g007.oni/0069_8bit.png', 'temp//frames/g007.oni/3509_16bit.png', 'temp//frames/g007.oni/3469_8bit.png', 'temp//frames/g007.oni/0357_color.png', 'temp//frames/g007.oni/3716_color.png', 'temp//frames/g007.oni/2201_8bit.png', 'temp//frames/g007.oni/0297_8bit.png', 'temp//frames/g007.oni/0252_color.png', 'temp//frames/g007.oni/1795_16bit.png', 'temp//frames/g007.oni/1172_8bit.png', 'temp//frames/g007.oni/0077_8bit.png', 'temp//frames/g007.oni/2644_16bit.png', 'temp//frames/g007.oni/2216_8bit.png', 'temp//frames/g007.oni/0458_color.png', 'temp//frames/g007.oni/1806_16bit.png', 'temp//frames/g007.oni/0262_8bit.png', 'temp//frames/g007.oni/2345_16bit.png', 'temp//frames/g007.oni/1093_16bit.png', 'temp//frames/g007.oni/3029_16bit.png', 'temp//frames/g007.oni/3523_16bit.png', 'temp//frames/g007.oni/3350_16bit.png', 'temp//frames/g007.oni/3006_16bit.png', 'temp//frames/g007.oni/0931_color.png', 'temp//frames/g007.oni/0539_color.png', 'temp//f